In [6]:
from transformers import DonutProcessor
import sys
sys.path.insert(1, 'PubTabNet/src/')
sys.path.insert(1, '../proj/src/')

from metric import TEDS
from processing_tabeleiro import TabeleiroProcessor
import json
import os


SPLIT = "val/"

PUBTABNET = 'data/anns/'

ANN_PATH = PUBTABNET

json_list = os.listdir(ANN_PATH + SPLIT)

aux_list = []
html_list = []

for f in json_list:
    if(f[-6:-5] == "L"):
        html_list.append(f)
    else:
        aux_list.append(f)
json_list = aux_list
processor = TabeleiroProcessor.from_pretrained("processors/Donut_PubTables_TML_Processor8k", crop_right=False)

In [7]:
with open("data/anns/test/final_eval.json") as f:
    final_eval = json.load(f)

In [34]:
count = 0

for i, f_name_n_table in enumerate(final_eval.items()):
    f_name, table = f_name_n_table
    try:
        table['html'].index('<b>')
        print(table['html'])
        break
        count += 1
    except ValueError:
        pass
print(count)

<html><body><table><thead><tr><td><b>Parameters</b></td><td><b>School A</b></td><td><b>School B</b></td></tr></thead><tbody><tr><td><b><i>q</i><sub><i>c</i></sub>, mean [ 95 <i>%</i> CI]</b></td><td><b>1.39×10<sup>−2</sup> [ 8.10×10<sup>−3</sup>−2.03×10<sup>−2</sup>]</b></td><td><b>1.96×10<sup>−2</sup> [ 1.11×10<sup>−2</sup>−2.89×10<sup>−2</sup>]</b></td></tr><tr><td><b><i>q</i><sub><i>g</i></sub>, mean [ 95 <i>%</i> CI]</b></td><td><b>4.36×10<sup>−3</sup> [ 9.61×10<sup>−4</sup>−8.34×10<sup>−3</sup>]</b></td><td><b>4.61×10<sup>−3</sup> [ 2.98×10<sup>−4</sup>−1.15×10<sup>−2</sup>]</b></td></tr><tr><td><b><i>q</i><sub><i>s</i></sub>, mean [ 95 <i>%</i> CI]</b></td><td><b>9.52×10<sup>−4</sup> [ 2.87×10<sup>−4</sup>−1.82×10<sup>−3</sup>]</b></td><td><b>2.96×10<sup>−3</sup> [ 1.64×10<sup>−3</sup>−4.45×10<sup>−3</sup>]</b></td></tr><tr><td><b><i>ε</i>, mean [ 95 <i>%</i> CI]</b></td><td><b>3.70×10<sup>−3</sup> [ 1.95×10<sup>−3</sup>−5.69×10<sup>−3</sup>]</b></td><td><b>2.65×10<sup>−3</sup> [

In [2]:
json_list[:10]

['PMC5771025_005_00.json',
 'PMC4077014_004_00.json',
 'PMC4042280_008_01.json',
 'PMC5629611_006_00.json',
 'PMC4824228_005_00.json',
 'PMC6060525_003_00.json',
 'PMC2265025_002_00.json',
 'PMC4846708_004_00.json',
 'PMC4100296_012_00.json',
 'PMC3112163_003_00.json']

In [31]:
gt_list = []
pred_list = []

teds = TEDS(n_jobs=4, structure_only = False, ignore_nodes='b')

In [32]:
processor.tokenizer("</s>")

{'input_ids': [0, 2, 2], 'attention_mask': [1, 1, 1]}

In [42]:
import random
import torch
from difflib import SequenceMatcher
from tqdm.auto import tqdm

removed_tags = []
soma_ratios = 0

for i, file_name in enumerate(tqdm(json_list)):

    with open(ANN_PATH + SPLIT + file_name) as f:
        annotation = json.load(f)
    with open(ANN_PATH + SPLIT + file_name[:-5]+"-HTML.json", encoding="utf-8") as f:
        gt_html = "<html><body><table>"+str(json.load(f))+"</table></body></html>"
    
    for tag in removed_tags:
        gt_html = gt_html.replace(tag, "")
    
    target_sequence = "<s>"+processor.json2token(annotation)+"</s>"
    input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=False,
        max_length= 2048,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)
    
    
    input_ids = torch.cat((input_ids, torch.Tensor([2, 2]).int()), 0)
    
    table = processor.token2ann( input_ids, 1)
    table_aux  = table.copy()
    
    
    pred_html = "<html><body><table>"+processor.table2html(table['tables'][0])+"</table></body></html>"
    
    pred_html = pred_html.replace('> ', '>')
    gt_html = gt_html.replace(" </td>", "</td>").replace("<td> ", "<td>").replace('"> ', '">')
    pred_list.append(pred_html)
    gt_list.append(gt_html)
    
    ratio = teds.evaluate(str(pred_html), str(gt_html))
    
    if(ratio < .9):
        print(processor.decode(input_ids[1]))
        #print(target_sequence)
        #print(table['tables'][0])
        print(pred_html)
        print()
        print(gt_html)
        print(ratio)
    if i% 100 == 0:   
        print(str(i)+'->', soma_ratios/max(1, i))
    
    soma_ratios += ratio
print(soma_ratios/len(json_list))

  0%|                                          | 1/9116 [00:00<55:12,  2.75it/s]

0-> 0.0


  0%|▏                                      | 38/9116 [01:01<7:46:32,  3.08s/it]

<table>
<html><body><table><thead><tr><td><b>Exposure</b></td><td><b>1st Trimester</b></td><td><b>2nd Trimester</b></td><td><b>3rd Trimester</b></td></tr></thead><tbody><tr><td><b>Spatiotemporally resolved PM<sub>2.5</sub></b></td><td></td><td></td><td></td></tr><tr><td><b><i>n</i></b></td><td>765</td><td>845</td><td>970</td></tr><tr><td><b>IQR (μg/m<sup>3</sup>)</b></td><td>2.29</td><td>1.97</td><td>2.24</td></tr><tr><td><b>β (95% CI)</b></td><td>–0.3 (–1.3, 0.7)</td><td>0.4 (–0.4, 1.2)</td><td>0.3 (–0.6, 1.2)</td></tr><tr><td><b>Temporally resolved PM<sub>2.5</sub></b></td><td></td><td></td><td></td></tr><tr><td><b><i>n</i></b></td><td>1,032</td><td>1,031</td><td>1,030</td></tr><tr><td><b>IQR (μg/m<sup>3</sup>)</b></td><td>2.35</td><td>1.77</td><td>2.05</td></tr><tr><td><b>β (95% CI)</b></td><td>0.1 (–0.9, 1.2)</td><td>0.1 (–0.7, 1.0)</td><td>0.5 (–0.4, 1.5)</td></tr><tr><td><b>Spatiotemporally resolved BC</b></td><td></td><td></td><td></td></tr><tr><td><b><i>n</i></b></td><td>1,099<

  1%|▍                                      | 97/9116 [02:23<1:57:55,  1.27it/s]

...

300-> 0.9957122397476833


  3%|█▎                                    | 310/9116 [07:27<2:45:12,  1.13s/it]

<table>
<html><body><table><thead><tr><td><b></b></td><td><b>All malignant pleural effusions</b></td><td><b>Sensitivity (%)</b></td><td><b>Sensitivity (%)</b></td><td><b>Malignant pleural mesothelioma only</b></td><td><b>Sensitivity (%)</b></td><td><b>Sensitivity (%)</b></td></tr><tr><td><b></b></td><td><b>Area under curve (95% CI)</b></td><td><b></b></td><td><b></b></td><td><b>Area under curve (95% CI)</b></td><td><b></b></td><td><b></b></td></tr></thead><tbody><tr><td>Cytology</td><td></td><td>51</td><td>100</td><td></td><td>31</td><td>100</td></tr><tr><td>DNA Integrity index cut-off: 1.02</td><td>0.766 (0.646 - 0.866)</td><td>57</td><td>90</td><td>0.823 (0.676-0.970)</td><td>75</td><td>90</td></tr><tr><td>Pleural fluid mesothelin cut-off: 12.72 nM</td><td>0.620 (0.480-0.759)</td><td>37</td><td>90</td><td>0.885 (0.774-0.997)</td><td>75</td><td>90</td></tr><tr><td>Serum mesothelin Cut-off: 1.34 nM</td><td>0.674 (0.546-0.802)</td><td>44</td><td>90</td><td>0.941 (0.869-1.00)</td><td>75<

  4%|█▌                                    | 371/9116 [08:42<3:25:17,  1.41s/it]


KeyboardInterrupt: 

In [ ]:
json.dump(pred_list, open("teste-pred.json", 'w'))
json.dump(gt_list, open("teste-gt.json", 'w'))

In [ ]:
a = '<thead><tr><td rowspan="2"><b>Cutoff (IRS)</b></td><td rowspan="2"><b>Number</b></td><td colspan="2"><b>Overall survival prognosis</b></td></tr><tr><td><b>Log-rank <i>P</i></b></td><td><b>Corrected <i>P</i><sup><i>∗</i></sup></b></td></tr></thead><tbody><tr><td>≥1</td><td>99/94</td><td>5.56</td><td>0.0184</td></tr><tr><td>≥2</td><td>129/64</td><td>4.37</td><td>0.0377</td></tr><tr><td>≥3</td><td>150/43</td><td>8.37</td><td>0.0038</td></tr><tr><td>≥4</td><td>161/32</td><td>6.35</td><td>0.0117</td></tr><tr><td>≥5 </td><td>177/16</td><td>4.50</td><td>0.0338</td></tr></tbody>'

for tag in ["<i>", "</i>","<b>","</b>"]:
    a = a.replace(tag, "")
    
a

In [ ]:
target_sequence = "Capítulo Primeiro Do título Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no[ 1 ] trem da Central um rapaz aqui do bairro, que eu conheço de vista e de cha- péu. Comprimentou-me, sentou-se ao pé de mim, falou da lua e dos mi- nistros, e acabou recitando-me versos. A viagem era curta, e os versos pode ser que não fossem inteiramente maus. Succedeu, porém, que, como eu es- tava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso. — Continue, disse eu acordando.[ 2 ] — Já acabei, murmurou ele.[ 3 ] — São muito bonitos.[ 4 ] Vi-lhe fazer um gesto para tirá-los outra vez do bolso, mas não passou[ 5 ] do gesto; estava amuado. No dia seguinte entrou a dizer de mim nomes feios, e acabou alcunhando-me Dom Casmurro."

input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=True,
        max_length= 1536,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)

In [ ]:
input_ids

In [ ]:
for token in input_ids:
    print(processor.decode(token))

In [ ]:
processor.decode(input_ids)